In [1]:
import pymp
from multiprocessing import Manager, cpu_count 
import sys
import os
import glob 
import gezi
from gezi import tqdm
import pandas as pd
import json
# import cudf
# import dask.dataframe as dd
import tensorflow as tf
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_string('f', None, '')
import melt

from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

#setting offilne
plotly.offline.init_notebook_mode(connected=True)

tensorflow_version: 1.14.0
torch_version: 1.3.1+cu100
Using TensorFlow backend.


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
# 载入模型
model_hour = '2020010112'
model_root = '/home/gezi/tmp/rank/data/tuwen_hour_sgsapp_v2/models/15/'
FLAGS(sys.argv)
melt.init(tf.Graph())
sess = melt.get_session()
model_path = os.path.realpath(f'{model_root}/{model_hour}')
print(model_path)
predictor = melt.Predictor(sess=sess)
predictor.restore(model_path)
assert predictor.graph.get_collection('index_feed')

2020-01-03 19:55:49 0:00:00 os.environ['CUDA_VISIBLE_DEVICES'] -1
2020-01-03 19:55:49 0:00:00 os.environ['CUDA_VISIBLE_DEVICES'] -1
2020-01-03 19:55:49 0:00:00 Tf dataset and Tf model train in Graph mode, support Horovod, keras False
2020-01-03 19:55:49 0:00:00 seed 71176
2020-01-03 19:55:49 0:00:00 log_level: 20
2020-01-03 19:55:50 0:00:00 valid_interval_epochs: 0.0
2020-01-03 19:55:50 0:00:00 buffer_size:0
2020-01-03 19:55:50 0:00:00 batch_size: 32 eval_batch_size: 32 batch_size_per_gpu: 32 num_gpus: 1 horovod: False
2020-01-03 19:55:50 0:00:00 model_dir /tmp/melt log_dir /tmp/melt


/search/odin/publicData/CloudS/chenghuige/rank/data/tuwen_hour_sgsapp_v2/models/15/2020010112
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /search/odin/publicData/CloudS/chenghuige/rank/data/tuwen_hour_sgsapp_v2/models/15/2020010112/model.ckpt-2020010112-1.00-3487


In [3]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:10000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [4]:
marks = ['tuwen', 'video']
head = 'mid,docid,dur,pred,abid,ori_lr_score,lr_score,show_time,position,video_time,impression_time,interests,rea,pred_click,pred_dur'
mid = 'e5e9b96d57c21de54eecf437e2b4f4c3f1c7de66b950'

In [5]:
def deal(df):
  df.hour = df.hour.astype(str)
  df = df.groupby(['abtest', 'hour'], as_index=False).last()
  df = df.sort_values(by=['hour'])
  df = df.drop_duplicates()
  df.hour = pd.to_datetime(df.hour, format='%Y%m%d%H')
  return df

# 遍历infos路径获取scores信息
def gen_df(mark, start_time=None, end_time=None):
  root = f'/home/gezi/tmp/rank/data/{mark}_hour_sgsapp_v2/infos/15'  
  dfs = Manager().dict()
  files = glob.glob(f'{root}/*/scores')
  with pymp.Parallel(cpu_count()) as p:
    for i in tqdm(p.range(len(files))):
      file = files[i]
      hour = os.path.basename(os.path.dirname(file))
      if not gezi.non_empty(file):
        continue
      df = pd.read_csv(file, sep='\t', header=None)
      df.columns = head.split(',')[:len(df.columns)]
      df = df[df.mid==mid]
      if len(df):
        df = df.assign(hour=hour)
        dfs[hour] = df
  print('num_hours_match:', len(dfs))
  df = pd.concat(dfs.values())
  df.name = mark
  return df

In [6]:
df_tuwen = gen_df('tuwen')

100%|██████████| 19/19 [01:54<00:00,  5.68s/it]

100%|██████████| 20/20 [02:45<00:00,  3.73s/it]


num_hours_match: 15


In [7]:
df = df_tuwen
df = df[df.ori_lr_score != -1.]
hour = '2020010113'
df = df[df.hour == hour]

In [8]:
from projects.feed.rank.utils.doc_info.KV import KV

In [9]:
kv = KV('110216', 'article_forward_index')

In [10]:
def get_title(docid):
  x = kv.mget([docid])
  x = json.loads(x[0])
  return x['title']

In [11]:
def is_cike(title):
  return '刺客' in title

In [12]:
df['title'] = df.docid.apply(get_title)

In [13]:
df['mark'] = df.title.apply(is_cike)

In [14]:
df = df.sort_values(by=['ori_lr_score'], ascending=False).reset_index()

In [15]:
cols = ['docid', 'abid', 'ori_lr_score', 'lr_score', 'pred', 'rea', 'impression_time', 'position', 'mark', 'hour']

In [16]:
df2 = df[df.mark==True]

In [17]:
df2[cols]

,docid,abid,ori_lr_score,lr_score,pred,rea,impression_time,position,mark,hour
6,20101d0x0TZMcx,13,0.212967,0.212967,0.205167,901,1577856955,6,True,2020010113
22,19c31b0x3Eu7Ya,13,0.159167,0.159167,0.127267,901,1577857014,8,True,2020010113
24,19c30h0x4BrFbL,13,0.157467,0.157467,0.114467,908,1577857093,4,True,2020010113
57,19c25g0x05Yhd0,13,0.122467,0.122467,0.104867,908,1577857307,6,True,2020010113
58,19c26f0x31MzTw,13,0.118167,0.118167,0.088567,908,1577857379,13,True,2020010113
76,19c27h0x252BKv,13,0.108667,0.108667,0.095667,908,1577857336,6,True,2020010113


In [18]:
df = df.sort_values(by=['ori_lr_score'], ascending=False)

In [19]:
pd.set_option('display.max_rows', 1000)

In [20]:
df[cols].sort_values(by=['impression_time', 'ori_lr_score'], ascending=False)

,docid,abid,ori_lr_score,lr_score,pred,rea,impression_time,position,mark,hour
92,open_doc_prod16599175,13,0.000000,0.000000,0.050367,0,1577857624,4,False,2020010113
96,20101b0x353NMC,13,0.000000,0.000000,0.020867,0,1577857624,0,False,2020010113
95,20101b0f2z7XI7,13,0.000000,0.000000,0.026967,0,1577857624,1,False,2020010113
94,20101b0x3vI5TH,13,0.000000,0.000000,0.030467,0,1577857624,2,False,2020010113
93,open_doc_prod16589543,13,0.000000,0.000000,0.028667,0,1577857624,3,False,2020010113
90,20200101A07WMV00,13,0.000000,0.000000,0.034867,0,1577857624,7,False,2020010113
89,open_doc_prod16598987,13,0.000000,0.000000,0.046967,0,1577857624,8,False,2020010113
88,20101b0x2sr4GX,13,0.000000,0.000000,0.052467,0,1577857624,9,False,2020010113
87,20200101A07U0700,13,0.000000,0.000000,0.052467,0,1577857624,10,False,2020010113
86,open_doc_prod16598409,13,0.000000,0.000000,0.039067,0,1577857624,15,False,2020010113


In [21]:
len(df[df.ori_lr_score==0.])

20

In [22]:
len(df)

102

In [23]:
tfrecord_root = '/home/gezi/tmp/rank/data/tuwen_hour_sgsapp_v2/tfrecords/'
tfrecord_dir = os.path.realpath(f'{tfrecord_root}/{hour}')
files = gezi.list_files(tfrecord_dir)

In [24]:
from projects.feed.rank.src.tfrecord_dataset import Dataset
from projects.feed.rank.src import util
num_records_file = os.path.join(tfrecord_dir, "num_records.txt")
total = melt.get_num_records(files) if not os.path.exists(num_records_file) else gezi.read_int_from(num_records_file)
print('total', total, file=sys.stderr)
batch_size = 512
l = []
with sess.graph.as_default():
  dataset = Dataset('valid')
  iter = dataset.make_batch(batch_size=batch_size, filenames=files, repeat=False)
  op = iter.get_next()

  print('---batch_size', dataset.batch_size, FLAGS.batch_size, melt.batch_size(), file=sys.stderr)  

  num_steps = -int(-total // batch_size)
  print('----num_steps', num_steps, file=sys.stderr) 
  try:
    for _ in tqdm(range(num_steps), total=num_steps, ascii=True):
      x, y = sess.run(op)
      mids = np.asarray([m.decode() for m in x['mid']])
      docids = np.asarray([m.decode() for m in x['docid']])
      x['mid'] = mids
      x['docid'] = docids
#       print(mids)
#       flter_flag = np.asarray([x == mid for x in mids])
#       print(filter_flag)
#       break
      if mid in mids:
#         for key in x:
#           x[key] = x[key][filter_flag]
        l += [x]
  except Exception:
    pass

total 1640581
---batch_size 512 32 32
----num_steps 3205
100%|##########| 3205/3205 [01:57<00:00, 27.39it/s]


In [25]:
import copy
l2 = copy.deepcopy(l)
for i in tqdm(range(len(l2))):
  filter_flag = l2[i]['mid'] == mid
#   filter_flag *= l2[i]['docid'] == '20101d0x0TZMcx'
  for key in l2[i]:
    try:
      l2[i][key] = l2[i][key][filter_flag]
    except Exception:
      pass
l2 = [x for x in l2 if len(x['mid'])]

100%|██████████| 107/107 [00:00<00:00, 1527.14it/s]


In [26]:
len(l)

107

In [27]:
def predict(x):
  feed_dict = {
                predictor.graph.get_collection('index_feed')[0]: x['index'],
                predictor.graph.get_collection('value_feed')[0]: x['value'],
                predictor.graph.get_collection('field_feed')[0]: x['field'],
                predictor.graph.get_collection('uid_feed')[0]: x['uid'],
                predictor.graph.get_collection('did_feed')[0]: x['did'],
                predictor.graph.get_collection('doc_idx_feed')[0]: x['history'],
            } 
  return predictor.predict(['pred', 'pred_click', 'pred_dur'], feed_dict)

In [28]:
predict(l2[0])

[array([[0.0441666]], dtype=float32),
 array([[0.00573766]], dtype=float32),
 array([[0.11539791]], dtype=float32)]

In [29]:
x = copy.deepcopy(l2[0])
predict(x)

[array([[0.0441666]], dtype=float32),
 array([[0.00573766]], dtype=float32),
 array([[0.11539791]], dtype=float32)]

In [30]:
def mask(x, fields):
  if isinstance(fields, str):
    fields = set(map(int, fields.split(',')))
  else:
    fields = set(fields)
  for i in range(len(x['field'])):
    flag = np.asarray([float(x !=0 and x not in fields) for x in x['field'][i]])
    x['value'][i] *= flag

In [31]:
field_names = open('/home/gezi/mine/pikachu/projects/feed/rank/src/conf/tuwen/fields.txt').readlines()
field_names = [f'{i+1}:{x.strip()}' for i, x in enumerate(field_names)]
field_names.insert(0, '0:None')
# field_names

In [32]:
res = []
for x in l2:
  pred = predict(x)
  res.append([x['docid'][0], x['impression_time'][0], x['position'][0], pred[0][0][0]])
df = pd.DataFrame.from_dict(res)
df.columns=['docid', 'impression_time', 'position', 'pred']
df.sort_values('pred', ascending=False)

,docid,impression_time,position,pred
70,open_doc_prod16562107,1577856955,4,0.282867
7,open_doc_prod16490069,1577857322,0,0.220867
87,2010120l2Nvk7L,1577857307,0,0.209867
5,19c25f0l0TO87r,1577857336,0,0.206267
77,20101d0x0TZMcx,1577856955,6,0.205167
101,19c31i0l1dphCw,1577857379,1,0.186367
88,open_doc_prod16542681,1577857379,5,0.183967
97,open_doc_prod16227053,1577857379,0,0.179667
16,19c3010l18Df7P,1577857409,0,0.175167
89,19c3040l0vejer,1577856955,5,0.173267


In [33]:
res = []
l2 = copy.deepcopy(l)
for x in l2:
  mask(x, [47,53,89,84,55])
  pred = predict(x)
  res.append([x['docid'][0], x['impression_time'][0], x['position'][0], pred[0][0][0]])
df = pd.DataFrame.from_dict(res)
df.columns=['docid', 'impression_time', 'position', 'pred']
df.sort_values('pred', ascending=False)

,docid,impression_time,position,pred
87,20191231A0CQGE00,1577852453,1,0.505267
1,19c30j0f1Z5YWd,1577747044,1,0.361567
31,20191230A083T500,1577852505,2,0.339167
55,open_doc_prod16562107,1577857052,0,0.317667
94,open_doc_prod16372945,1577856595,1,0.301267
13,19c2690l1xvJqk,1577855321,1,0.289267
67,19c27g0x3N3ZLA,1577857968,1,0.287867
42,open_doc_prod16569633,1577856929,1,0.284267
25,open_doc_prod16543021,1577856491,1,0.265067
53,19c30d0l25EF7A,1577855911,1,0.260767


In [34]:
res = []
for i in range(91):
  x = copy.deepcopy(l2[1])
  mask(x, [i])
  res.append([field_names[i], predict(x)[0][0][0]])
df = pd.DataFrame.from_dict(res)
df.columns=['field', 'pred']
df.sort_values('pred')

,field,pred
18,18:IATKWSE,0.289367
21,21:IATSO,0.330967
4,4:CITATCOTP,0.350367
24,24:IAVGDUR,0.350867
73,73:XATSWCL,0.351267
11,11:DATRD,0.352767
23,23:IATVI,0.354167
20,20:IATOR,0.354167
19,19:IATLO,0.356767
37,37:ILOC,0.356767


In [35]:
x = copy.deepcopy(l2[0])
mask(x, [47,53,89,84,55])
predict(x)

[array([[0.12986659],
        [0.24986659],
        [0.1806666 ],
        [0.21856658],
        [0.0924666 ],
        [0.15186659],
        [0.21236658],
        [0.4085666 ],
        [0.14836659],
        [0.12916659],
        [0.19306658],
        [0.1769666 ],
        [0.0990666 ],
        [0.1237666 ],
        [0.0895666 ],
        [0.14746659],
        [0.16496658],
        [0.0314666 ],
        [0.13776658],
        [0.1285666 ],
        [0.19886659],
        [0.3365666 ],
        [0.0809666 ],
        [0.0760666 ],
        [0.2680666 ],
        [0.1226666 ],
        [0.13886659],
        [0.3840666 ],
        [0.2655666 ],
        [0.3263666 ],
        [0.12816659],
        [0.15996659],
        [0.2078666 ],
        [0.18696658],
        [0.18836659],
        [0.21656659],
        [0.1155666 ],
        [0.20556659],
        [0.1358666 ],
        [0.12296659],
        [0.0657666 ],
        [0.14996658],
        [0.2708666 ],
        [0.1277666 ],
        [0.3964666 ],
        [0